<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Computer%20Vision/Flower_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Importing Dataset

In [39]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-01-24 15:35:28--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-24 15:35:28 (79.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [4]:
from helper_functions import create_tensorboard_callback,plot_loss_curves,unzip_data,walk_through_dir

In [5]:
!pip install tensorflow-datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import tensorflow_datasets as tfds
import pathlib
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')

228813984/228813984 [==============================] - 6s 0us/step


In [7]:
data_dir

PosixPath('/root/.keras/datasets/flower_photos')

## 2. Setting Up Data Loaders

In [8]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_dataset = image_dataset_from_directory(data_dir,label_mode = "categorical",image_size = (224,224),validation_split = 0.2,subset = "training",seed = 42)
test_dataset = image_dataset_from_directory(data_dir,label_mode = "categorical",image_size = (224,224),validation_split = 0.2,subset = "validation",seed = 32)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.
Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [9]:
class_names = train_dataset.class_names

## 3. Data Augmentation

In [51]:
from tensorflow.keras.layers.experimental import preprocessing
data_augmentation = tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal",input_shape=(224, 224, 3)),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.Rescaling(1./255),
    
])

## 2. Model 0 Simple Model

In [47]:
checkpoint_path ="model_0_weight_checkpoints/checkpoint.ckpt"
     
checkpoint_call_back = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    save_weights_only = True,
    save_best_only = True,
    save_freq = "epoch"
)

In [72]:
model_0 = tf.keras.Sequential([
    data_augmentation,
    tf.keras.layers.Conv2D(
        filters = 32,
        kernel_size = 3,
        strides  = (1,1),
        activation = "relu",name = "input_layer"
        ),
    tf.keras.layers.Conv2D(
        32,3,activation = "relu"
    ),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Conv2D(
        32,3,activation = "relu"
    ),
    tf.keras.layers.MaxPool2D(2),
    tf.keras.layers.Conv2D(
        32,3,activation = "relu"
    ),
    tf.keras.layers.GlobalMaxPool2D(),
    tf.keras.layers.Dense(len(class_names),activation = "softmax",name = "output_layer"),

])
model_0.compile(loss = "categorical_crossentropy",metrics = ["accuracy"],optimizer = "adam")

In [ ]:
history_0 = model_0.fit(
    train_dataset,epochs = 5,
    steps_per_epoch = len(train_dataset),
    validation_data = test_dataset,
    validation_steps = len(test_dataset),
    callbacks = [checkpoint_call_back]
)

Epoch 1/5
32/92 [=========>....................] - ETA: 1:14 - loss: 1.6092 - accuracy: 0.2246

In [ ]:
plot_loss_curves